# 1. Data Preprocessing

> ## Contents  
  * 0) 버스정류장별, 측정소별 데이터를 만들기 위해 공통적으로 사용되어야 하는 정보  
  * 1) 측정소별 데이터  
  * 2) 버스정류장별 데이터  
  * 3) 모델 훈련데이터 / 예측데이터

In [ ]:
import pathlib
from geoband import API

import numpy as np
import pandas as pd
import re

import geopandas as gpd
import shapely
from shapely.geometry import MultiPolygon, Polygon, Point

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
input_path = pathlib.Path('./input')
if not input_path.is_dir():
    input_path.mkdir()

API.GetCompasData('SBJ_2102_002', '1', input_path.joinpath('1.수원시_버스정류장.csv'))
API.GetCompasData('SBJ_2102_002', '2', input_path.joinpath('2.수원시_버스정류장별_승하차이력(1).csv'))
API.GetCompasData('SBJ_2102_002', '3', input_path.joinpath('3.수원시_버스정류장별_승하차이력(2).csv'))
API.GetCompasData('SBJ_2102_002', '4', input_path.joinpath('4.수원시_버스정류장별_승하차이력(3).csv'))
API.GetCompasData('SBJ_2102_002', '5', input_path.joinpath('5.수원시_버스정류장별_승하차이력(4).csv'))
API.GetCompasData('SBJ_2102_002', '6', input_path.joinpath('6.수원시_버스정류장별_승하차이력(5).csv'))
API.GetCompasData('SBJ_2102_002', '7', input_path.joinpath('7.수원시_버스정류장별_노선현황.csv'))
API.GetCompasData('SBJ_2102_002', '8', input_path.joinpath('8.수원시_지하철역_위치정보.csv'))
API.GetCompasData('SBJ_2102_002', '9', input_path.joinpath('9.수원시_지하철역별_이용현황(2017~2019).csv'))
API.GetCompasData('SBJ_2102_002', '10', input_path.joinpath('10.수원시_옥외광고물현황.csv'))
API.GetCompasData('SBJ_2102_002', '11', input_path.joinpath('11.수원시_대기오염도_측정현황.csv'))
API.GetCompasData('SBJ_2102_002', '12', input_path.joinpath('12.수원시_주차장현황.csv'))
API.GetCompasData('SBJ_2102_002', '13', input_path.joinpath('13.수원시_기상데이터(2020).csv'))
API.GetCompasData('SBJ_2102_002', '14',  input_path.joinpath('14.수원시_시간대별_유동인구(2020).csv'))
API.GetCompasData('SBJ_2102_002', '15',  input_path.joinpath('15.수원시_성연령별_유동인구(2020).csv'))
API.GetCompasData('SBJ_2102_002', '16',  input_path.joinpath('16.수원시_요일별_유동인구(2020).csv'))
API.GetCompasData('SBJ_2102_002', '17',  input_path.joinpath('17.수원시_인구정보(고령)_격자.geojson'))
API.GetCompasData('SBJ_2102_002', '18',  input_path.joinpath('18.수원시_인구정보(생산가능)_격자.geojson'))
API.GetCompasData('SBJ_2102_002', '19',  input_path.joinpath('19.수원시_인구정보(유소년)_격자.geojson'))
API.GetCompasData('SBJ_2102_002', '20',  input_path.joinpath('20.수원시_교통노드.geojson'))
API.GetCompasData('SBJ_2102_002', '21',  input_path.joinpath('21.수원시_교통링크.geojson'))
API.GetCompasData('SBJ_2102_002', '22',  input_path.joinpath('22.수원시_상세도로망_LV6.geojson'))
API.GetCompasData('SBJ_2102_002', '23',  input_path.joinpath('23.수원시_평일_일별_시간대별_추정교통량_LV6.csv'))
API.GetCompasData('SBJ_2102_002', '24',  input_path.joinpath('24.수원시_평일_일별_혼잡빈도강도_LV6.csv'))
API.GetCompasData('SBJ_2102_002', '25',  input_path.joinpath('25.수원시_평일_일별_혼잡시간강도_LV6.csv'))
API.GetCompasData('SBJ_2102_002', '26',  input_path.joinpath('26.수원시_인도(2017).geojson'))
API.GetCompasData('SBJ_2102_002', '27',  input_path.joinpath('27.수원시_도로명주소(건물).geojson'))
API.GetCompasData('SBJ_2102_002', '28',  input_path.joinpath('28.수원시_건물연면적_격자.geojson'))
API.GetCompasData('SBJ_2102_002', '29',  input_path.joinpath('29.수원시_법정경계(시군구).geojson'))
API.GetCompasData('SBJ_2102_002', '30',  input_path.joinpath('30.수원시_법정경계(읍면동).geojson'))
API.GetCompasData('SBJ_2102_002', '31',  input_path.joinpath('31.수원시_행정경계(읍면동).geojson'))
API.GetCompasData('SBJ_2102_002', '32',  input_path.joinpath('32.수원시_지적도.geojson'))
API.GetCompasData('SBJ_2102_002', '33',  input_path.joinpath('33.코드정의서.xlsx'))

for path in list(input_path.glob('*.csv')) + list(input_path.glob('*.geojson')):
    print(path)

input/1.수원시_버스정류장.csv
input/2.수원시_버스정류장별_승하차이력(1).csv
input/3.수원시_버스정류장별_승하차이력(2).csv
input/4.수원시_버스정류장별_승하차이력(3).csv
input/5.수원시_버스정류장별_승하차이력(4).csv
input/6.수원시_버스정류장별_승하차이력(5).csv
input/7.수원시_버스정류장별_노선현황.csv
input/8.수원시_지하철역_위치정보.csv
input/9.수원시_지하철역별_이용현황(2017~2019).csv
input/10.수원시_옥외광고물현황.csv
input/11.수원시_대기오염도_측정현황.csv
input/12.수원시_주차장현황.csv
input/13.수원시_기상데이터(2020).csv
input/14.수원시_시간대별_유동인구(2020).csv
input/15.수원시_성연령별_유동인구(2020).csv
input/16.수원시_요일별_유동인구(2020).csv
input/23.수원시_평일_일별_시간대별_추정교통량_LV6.csv
input/24.수원시_평일_일별_혼잡빈도강도_LV6.csv
input/25.수원시_평일_일별_혼잡시간강도_LV6.csv
input/df_버스정류장별_고정데이터.csv
input/정류장_유동인구_요일.csv
input/유동인구_예측.csv
input/GGD_RouteInfo_M.csv
input/17.수원시_인구정보(고령)_격자.geojson
input/18.수원시_인구정보(생산가능)_격자.geojson
input/19.수원시_인구정보(유소년)_격자.geojson
input/20.수원시_교통노드.geojson
input/21.수원시_교통링크.geojson
input/22.수원시_상세도로망_LV6.geojson
input/26.수원시_인도(2017).geojson
input/27.수원시_도로명주소(건물).geojson
input/28.수원시_건물연면적_격자.geojson
input/29.수원시_법정경계(시군구).geojson
input/30.수원시_법정경계(

In [ ]:
# 데이터 입력
# 데이터 입력 시 용이성을 위해 df_##으로 이름 지정

df_01= pd.read_csv("input/1.수원시_버스정류장.csv")
df_02= pd.read_csv("input/2.수원시_버스정류장별_승하차이력(1).csv")
df_03= pd.read_csv("input/3.수원시_버스정류장별_승하차이력(2).csv")
df_04= pd.read_csv("input/4.수원시_버스정류장별_승하차이력(3).csv")
df_05= pd.read_csv("input/5.수원시_버스정류장별_승하차이력(4).csv")
df_06= pd.read_csv("input/6.수원시_버스정류장별_승하차이력(5).csv")
df_07= pd.read_csv("input/7.수원시_버스정류장별_노선현황.csv")
df_08= pd.read_csv("input/8.수원시_지하철역_위치정보.csv")
df_09= pd.read_csv("input/9.수원시_지하철역별_이용현황(2017~2019).csv")
df_10= pd.read_csv("input/10.수원시_옥외광고물현황.csv")
df_11= pd.read_csv("input/11.수원시_대기오염도_측정현황.csv")
df_12= pd.read_csv("input/12.수원시_주차장현황.csv")
df_13= pd.read_csv("input/13.수원시_기상데이터(2020).csv")
df_14= pd.read_csv("input/14.수원시_시간대별_유동인구(2020).csv")
df_15= pd.read_csv("input/15.수원시_성연령별_유동인구(2020).csv")
df_16= pd.read_csv("input/16.수원시_요일별_유동인구(2020).csv")
df_17= gpd.read_file("input/17.수원시_인구정보(고령)_격자.geojson")
df_18= gpd.read_file("input/18.수원시_인구정보(생산가능)_격자.geojson")
df_19= gpd.read_file("input/19.수원시_인구정보(유소년)_격자.geojson")
df_20= gpd.read_file("input/20.수원시_교통노드.geojson")
df_21= gpd.read_file("input/21.수원시_교통링크.geojson")
df_22= gpd.read_file("input/22.수원시_상세도로망_LV6.geojson")
df_23= pd.read_csv("input/23.수원시_평일_일별_시간대별_추정교통량_LV6.csv")
df_24= pd.read_csv("input/24.수원시_평일_일별_혼잡빈도강도_LV6.csv")
df_25= pd.read_csv("input/25.수원시_평일_일별_혼잡시간강도_LV6.csv")
df_26= gpd.read_file("input/26.수원시_인도(2017).geojson")
df_27= gpd.read_file("input/27.수원시_도로명주소(건물).geojson")
df_28= gpd.read_file("input/28.수원시_건물연면적_격자.geojson")
df_29= gpd.read_file("input/29.수원시_법정경계(시군구).geojson")
df_30= gpd.read_file("input/30.수원시_법정경계(읍면동).geojson")
df_31= gpd.read_file("input/31.수원시_행정경계(읍면동).geojson")
df_32= gpd.read_file("input/32.수원시_지적도.geojson")

In [ ]:
# coloumn name 수정

df_01.rename(columns={'정류장ID':'정류소ID'}, inplace=True) # df_02 ~ df_06과 동일하게
df_09.rename(columns={'역명':'역사명', '호선명':'노선명'}, inplace=True) # df_08과 동일하게

# 0) 측정소와 버스정류장 공통

## a) 대기환경 / 날씨 정보

* 대기환경 데이터 : 2020년 수원시 시간별 대기질 측정정보 / `2020년수원미세먼지.csv` /  출처 : [경기데이터드림](https://data.gg.go.kr/portal/data/village/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=GE0DUHTX3VX0GL4R0LUS26448884&infSeq=1&order=&sigunNm=%EC%88%98%EC%9B%90%EC%8B%9C&sigunFlag=41110&searchWord=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80)  
* 날씨 데이터 : `13.수원시_기상데이터(2020).csv`


In [ ]:
# ============== 대기질 측정정보 데이터 ============

df_2020 = pd.read_csv("2020년수원미세먼지.csv", encoding='CP949') # 시간별 대기질 측정정보 데이터
df_2020['측정소명'] = df_2020['측정소명'].apply(lambda x : '동수원' if x=='경수대로(동수원)' else x) # 편의를 위해 측정소명 변경
df_2020['측정일시각'] = pd.to_datetime(df_2020['측정일시각']) # Date 변수로 전환
df_2020['날짜'] = pd.to_datetime(pd.to_datetime(df_2020['측정일시각']).dt.date) # 날짜 추출

# 시간별 미세먼지 데이터를 일평균 미세먼지 데이터로 변환
df_미세 = pd.DataFrame.reset_index(pd.pivot_table(df_2020, index=['날짜'], columns='측정소명', values='미세먼지PM10농도값',aggfunc = 'mean')) 
# 시간별 초미세먼지 데이터를 일평균 초미세먼지 데이터로 변환
df_초미세 = pd.DataFrame.reset_index(pd.pivot_table(df_2020, index=['날짜'], columns='측정소명', values='미세먼지PM2.5농도값',aggfunc = 'mean'))


# ============== 날씨 데이터 ============

df_13['일시'] = pd.to_datetime(df_13['일시'])  # Date 변수로 전환
df_13['날짜'] = pd.to_datetime(pd.to_datetime(df_13['일시']).dt.date)  # 날짜 추출

# 기온, 습도, 풍속(최고, 최저) 
df_max = df_13.groupby(['날짜'], as_index=False).max()[['날짜', '기온(°C)', '풍속(m/s)','습도(%)']]
df_max.rename(columns = {'기온(°C)': '최고기온','풍속(m/s)':'최고풍속','습도(%)': '최고습도' }, inplace = True)
df_min = df_13.groupby(['날짜'], as_index=False).min()[['날짜', '기온(°C)', '풍속(m/s)','습도(%)']]
df_min.rename(columns = {'기온(°C)': '최저기온','풍속(m/s)':'최저풍속','습도(%)': '최저습도' }, inplace = True)

## b) 건물 정보  
  
* 건물 코드와 그에 따른 건물 종류(=코드명) 정보가 들어있는 데이터 : `33.코드정의서.xlsx`  
* 건물 데이터 : `27.수원시_도로명주소(건물).geojson`  
* 지하철역 데이터 : `input/8.수원시_지하철역_위치정보.csv`

In [ ]:
# 건물 코드와 종류에 대한 정보 가져오기
코드정의서 = pd.read_excel("input/33.코드정의서.xlsx")
building = 코드정의서[521:849].reset_index(drop=True) # 건물용도코드만 분리
building.columns = ['BDTYP_CD','코드명']

# 건물 코드가 잘못 들어가 있는 경우 수정
df_27.loc[(df_27['BDTYP_CD']=='07199'),'BDTYP_CD']="06201"
df_27.loc[(df_27['BDTYP_CD']=='08203'),'BDTYP_CD']="08003"

# 건물 이름도 없고, 종류 또한 알 수 없어 제거
unknown = ["04505", "03024", "07108", "08800"]
df_27 = df_27[df_27['BDTYP_CD'].isin(unknown) == False].reset_index(drop=True)

# 건물 코드와 그에 따른 건물 종류(=코드명)부여
df_27['코드명']=0

for i in range(len(df_27)):
    for j in range(len(building)):
        if df_27['BDTYP_CD'][i]==building['BDTYP_CD'][j]:
            df_27['코드명'][i]=building['코드명'][j]

# 건물 종류별 데이터셋 생성

## 학교
school = df_27[df_27['코드명'].str.contains('학교')==True][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']].reset_index(drop=True)
## 병원
hospi = df_27[(df_27['코드명'].str.contains('병원')==True)|(df_27['코드명'].str.contains('한의원')==True)|(df_27['코드명'].str.contains('의원')==True)|(df_27['코드명'].str.contains('기타의료시설')==True)][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']].reset_index(drop=True)
## 주거용
house = df_27[(df_27['코드명'].str.contains('주택')==True)|(df_27['코드명'].str.contains('아파트')==True)|(df_27['코드명'].str.contains('고시원')==True)|(df_27['코드명'].str.contains('오피스텔')==True)][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']].reset_index(drop=True)
## 회사
company = df_27[(df_27['코드명'].str.contains('기타공공업무시설')==True)|(df_27['코드명'].str.contains('일반공공업무시설')==True)|(df_27['코드명'].str.contains('사무소')==True)|(df_27['코드명'].str.contains('기타일반업무시설')==True)|(df_27['코드명'].str.contains('기타사무소')==True)][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']].reset_index(drop=True)
## 공장
factory = df_27[(df_27['코드명'].str.contains('일반공장')==True)|(df_27['코드명'].str.contains('정비공장')==True)|(df_27['코드명'].str.contains('대형점')==True)][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']] .reset_index(drop=True)
## 상업 시설
store = df_27[(df_27['코드명'].str.contains('시장')==True)|(df_27['코드명'].str.contains('백화점')==True)|(df_27['코드명'].str.contains('대형판매점')==True)][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']].reset_index(drop=True)
## 관공서
public = df_27[(df_27['코드명'].str.contains('동사무소')==True)|(df_27['코드명'].str.contains('도서관')==True)|(df_27['코드명'].str.contains('우체국')==True)|(df_27['코드명'].str.contains('파출소')==True)|(df_27['코드명'].str.contains('경찰서')==True)|(df_27['코드명'].str.contains('소방서')==True)|(df_27['코드명'].str.contains('국가기관청사')==True)][['BDTYP_CD','BULD_NM','EMD_CD','코드명','geometry']].reset_index(drop=True)


# 각 건물의 중심점(=geo_cent)을 찾아주는 함수
def make_geocent(data):
    res = data['geometry'].apply(lambda x : list(x[0].centroid.coords[0]))
    res = res.apply(lambda x : Point(x))
    return res

school['geo_cent'] = make_geocent(school)
hospi['geo_cent'] = make_geocent(hospi)
house['geo_cent'] = make_geocent(house)
company['geo_cent'] = make_geocent(company)
factory['geo_cent'] = make_geocent(factory)
store['geo_cent'] = make_geocent(store)
public['geo_cent'] = make_geocent(public)
df_08['geo_cent'] = list(Point([df_08['lon'][i], df_08['lat'][i]]) for i in range(len(df_08))) # 지하철역

## c) 교통량 정보  
  
* 도로 정보 데이터 : `22.수원시_상세도로망_LV6.geojson`
* 도로별 교통량 데이터 : `23.수원시_평일_일별_시간대별_추정교통량_LV6.csv`

In [ ]:
df_23['link_id'] = round(df_23['상세도로망_LinkID']/100) # 상행, 하행 정보를 포함하지 않은 link id
df_23 = df_23[df_23['시간적범위']!='fulltime'] # fulltime 자료 제외
df_23['시간적범위'] = df_23['시간적범위'].astype(int)

# 일별 / 시간별 데이터를 일평균 데이터로 변환
df_교통량_일평균 = df_23.groupby(['link_id','시간적범위'], as_index=False).mean()[['link_id','전체_추정교통량','시간적범위']] 
df_교통량_일평균.rename(columns={'시간적범위':'시간'}, inplace=True)

# 교통량이 가장 적은 시간대(=12시)와 많은 시간대(=17시)의 값만 이용
df_교통량_일평균_1217 = pd.DataFrame.reset_index(df_교통량_일평균[(df_교통량_일평균['시간']==12)| (df_교통량_일평균['시간']==17)]).drop('index', axis=1)
df_교통량_일평균_1217 = pd.DataFrame.reset_index(pd.pivot_table(df_교통량_일평균_1217, index=['link_id'], columns='시간', values='전체_추정교통량'))

# 교통량 정보가 존재하는 도로만 df_도로위치 데이터셋에 포함시키기
df_교통량_일별총합 = df_23.groupby(['link_id'], as_index=False).sum()
df_22['link_id'] = df_22['link_id'].astype(int)
df_도로위치 = pd.merge(df_22[['link_id','geometry']], df_교통량_일별총합['link_id'], on='link_id', how='inner')

# 1) 측정소별 데이터셋 생성  
: 버스데이터별 미세먼지/초미세먼지를 예측하는 모델을 훈련시키기 위한 데이터셋

## a) 측정소 정보 데이터 
대기정보 측정소 데이터 : `11.수원시_대기오염도_측정현황.csv`

In [ ]:
# 측정소 이름, 위경도 정보
df_측정소 = df_11[['측정소', 'lon', 'lat']][:8]
df_측정소 = df_측정소.sort_values(by='측정소').reset_index(drop=True)
df_측정소['geo_cent'] = df_측정소.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
df_측정소

,측정소,lon,lat,geo_cent
0,고색동,126.976384,37.252329,POINT (126.976384 37.252329)
1,광교동,127.070615,37.291321,POINT (127.070615 37.291321)
2,동수원,127.030494,37.276744,POINT (127.030494 37.276744)
3,신풍동,127.010465,37.283850,POINT (127.010465 37.28385)
4,영통동,127.056294,37.246901,POINT (127.056294 37.246901)
5,인계동,127.028723,37.263637,POINT (127.028723 37.263637)
6,천천동,126.975305,37.292877,POINT (126.975305 37.292877)
7,호매실동,126.957008,37.266153,POINT (126.957008 37.266153)


In [ ]:
# 측정소 반경 500m buffer 
buffer_500측정소= gpd.GeoDataFrame(df_측정소['geo_cent'], geometry='geo_cent')
buffer_500측정소.crs = {'init':'epsg:4326'} # 좌표계가 지정되지 않아 알맞는 4326좌표계를 설정해줍니다.
buffer_500측정소= buffer_500측정소.to_crs({'init':'epsg:5179'})  # 4326좌표계는 미터단위 분석이 불가능하므로 100미터 버퍼 생성이 가능한 5179좌표계로 변환합니다.
buffer_500측정소=buffer_500측정소.buffer(500)
buffer_500측정소=buffer_500측정소.to_crs({'init':'epsg:4326'}) # 버퍼 후 4326좌표계로 되돌아옵니다.

In [ ]:
## 측정소 반경 1000m buffer 
buffer_1000측정소= gpd.GeoDataFrame(df_측정소['geo_cent'], geometry='geo_cent')
buffer_1000측정소.crs = {'init':'epsg:4326'}  # 좌표계가 지정되지 않아 알맞는 4326좌표계를 설정해줍니다.
buffer_1000측정소= buffer_1000측정소.to_crs({'init':'epsg:5179'}) # 4326좌표계는 미터단위 분석이 불가능하므로 100미터 버퍼 생성이 가능한 5179좌표계로 변환합니다.
buffer_1000측정소=buffer_1000측정소.buffer(1000)
buffer_1000측정소=buffer_1000측정소.to_crs({'init':'epsg:4326'}) # 버퍼 후 4326좌표계로 되돌아옵니다.

## b) 미세먼지/초미세먼지 측정량을 이용해 측정소별 보간데이터 생성 후, 날씨 데이터와 병합
* 대기환경 데이터 : 2020년 수원시 시간별 대기질 측정정보 / `2020년수원미세먼지.csv` /  출처 : [경기데이터드림](https://data.gg.go.kr/portal/data/village/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=GE0DUHTX3VX0GL4R0LUS26448884&infSeq=1&order=&sigunNm=%EC%88%98%EC%9B%90%EC%8B%9C&sigunFlag=41110&searchWord=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80)  
* 날씨 데이터 : `13.수원시_기상데이터(2020).csv`


### b-1) 미세먼지 

In [ ]:
# ===== 미세먼지 예측 모델 훈련을 위한 측정소별 미세먼지 보간데이터 생성 =====

df_측정소거리 = df_측정소[['측정소','geo_cent']]
df_측정소거리[df_측정소거리['측정소']]=0

# 측정소간 거리계산
for i in range(len(df_측정소거리)):
    for j in range(len(df_측정소거리)):
        df_측정소거리.iloc[i,j+2] = df_측정소거리['geo_cent'][i].distance(df_측정소거리['geo_cent'][j])
        
# 측정소별 보간데이터 계산을 위한 데이터셋 생성
df_미세_ = pd.concat([df_미세]*len(df_측정소)).reset_index(drop=True)
df_미세_['측정소']=0

for i in range(len(df_측정소)):
    df_미세_['측정소'][i*len(df_미세):(i+1)*len(df_미세)] = df_측정소['측정소'][i]

df_기존 = pd.merge(df_미세_, df_측정소거리, on='측정소', how="left")
df_기존 = df_기존.dropna(axis=0).reset_index(drop=True) # 측정기록이 없는 일자는 삭제

df_측정소_보간데이터 = df_기존[['날짜','측정소','geo_cent']]
df_측정소_보간데이터['target'] = float(0)
for i in range(len(df_기존)):
    for j in range(1,9):
        if df_기존['측정소'][i] == df_기존.columns[j].split('_')[0]:
            df_측정소_보간데이터['target'][i] = df_기존.iloc[i,j] # target = 일평균 미세먼지 지수

# 측정소별 보간데이터 입력
df_측정소_보간데이터['보간데이터_4'] = float(0)
for i in range(len(df_기존)):
    order = np.array(df_기존.iloc[i,13:]).argsort().tolist()
    b_4 = sum(1/np.array(df_기존.iloc[i,13:])[order][1:5])
    a_4 = sum(1/np.array(df_기존.iloc[i,13:])[order][1:5] * np.array(df_기존.iloc[i,1:9])[order][1:5])
    df_측정소_보간데이터['보간데이터_4'][i] = float(a_4/ b_4)
    
# 날씨 데이터와 병합
df_측정소_보간데이터 = pd.merge(df_측정소_보간데이터, df_max[['날짜','최고기온','최고풍속', '최고습도']], on='날짜', how="left")
df_측정소_보간데이터 = pd.merge(df_측정소_보간데이터, df_min[['날짜','최저기온','최저풍속', '최저습도']], on='날짜', how="left")
df_측정소_보간데이터 = df_측정소_보간데이터.dropna(axis=0).reset_index(drop=True)
df_측정소_보간데이터

,날짜,측정소,geo_cent,target,보간데이터_4,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도
0,2020-01-01,고색동,POINT (126.976384 37.252329),30.173913,32.259131,0.2,2.4,90.0,-6.3,0.2,64.0
1,2020-01-02,고색동,POINT (126.976384 37.252329),51.869565,58.040780,3.2,1.4,92.0,-0.3,0.1,76.0
2,2020-01-03,고색동,POINT (126.976384 37.252329),63.652174,69.981241,5.2,3.0,99.0,-3.9,0.0,51.0
3,2020-01-04,고색동,POINT (126.976384 37.252329),51.260870,56.669166,7.4,3.4,100.0,-4.3,0.1,48.0
4,2020-01-05,고색동,POINT (126.976384 37.252329),44.416667,50.191075,6.2,1.7,96.0,-5.5,0.3,43.0
...,...,...,...,...,...,...,...,...,...,...,...
2699,2020-12-27,호매실동,POINT (126.957008 37.266153),71.166667,69.173530,8.9,2.9,93.0,0.4,0.0,55.0
2700,2020-12-28,호매실동,POINT (126.957008 37.266153),75.708333,69.596279,10.8,2.4,98.0,1.8,0.0,51.0
2701,2020-12-29,호매실동,POINT (126.957008 37.266153),68.695652,67.394051,3.9,5.0,100.0,-4.5,0.1,56.0
2702,2020-12-30,호매실동,POINT (126.957008 37.266153),28.409091,32.660328,-5.4,6.4,68.0,-12.1,2.4,44.0


In [ ]:
df_측정소_보간데이터.to_csv('df_측정소_보간데이터.csv', index=False)

### b-2) 초미세먼지

In [ ]:
# ===== 초미세먼지 예측 모델 훈련을 위한 측정소별 미세먼지 보간데이터 생성 =====

df_측정소거리 = df_측정소[['측정소','geo_cent']]
df_측정소거리[df_측정소거리['측정소']] = 0

# 측정소간 거리계산
for i in range(len(df_측정소거리)):
    for j in range(len(df_측정소거리)):
        df_측정소거리.iloc[i,j+2] = df_측정소거리['geo_cent'][i].distance(df_측정소거리['geo_cent'][j])

# 측정소별 보간데이터 계산을 위한 데이터셋 생성
df_초미세_ = pd.DataFrame.reset_index(pd.concat([df_초미세]*len(df_측정소))).drop(['index'], axis=1)
df_초미세_['측정소'] = 0
for i in range(len(df_측정소)):
    df_초미세_['측정소'][i*len(df_초미세):(i+1)*len(df_초미세)] = df_측정소['측정소'][i]
df_기존 = pd.merge(df_초미세_, df_측정소거리, on='측정소', how="left")
df_기존 = df_기존.dropna(axis=0).reset_index(drop=True)
df_측정소_보간데이터2 = df_기존[['날짜','측정소','geo_cent']]
df_측정소_보간데이터2['target'] = float(0)
for i in range(len(df_기존)):
    for j in range(1,9):
        if df_기존['측정소'][i] == df_기존.columns[j].split('_')[0]:
            df_측정소_보간데이터2['target'][i] = df_기존.iloc[i,j]

# 측정소별 보간데이터 입력
df_측정소_보간데이터2['보간데이터_4'] = float(0)
for i in range(len(df_기존)):
    order = np.array(df_기존.iloc[i,13:]).argsort().tolist()
    b_4 = sum(1/np.array(df_기존.iloc[i,13:])[order][1:5])
    a_4 = sum(1/np.array(df_기존.iloc[i,13:])[order][1:5] * np.array(df_기존.iloc[i,1:9])[order][1:5])
    df_측정소_보간데이터2['보간데이터_4'][i] = float(a_4/ b_4)
    
# 날씨 데이터와 병합
df_측정소_보간데이터2 = pd.merge(df_측정소_보간데이터2, df_max[['날짜','최고기온','최고풍속', '최고습도']], on='날짜', how="left")
df_측정소_보간데이터2 = pd.merge(df_측정소_보간데이터2, df_min[['날짜','최저기온','최저풍속', '최저습도']], on='날짜', how="left")
df_측정소_보간데이터2 = df_측정소_보간데이터2.dropna(axis=0).reset_index(drop=True)
df_측정소_보간데이터2

,날짜,측정소,geo_cent,target,보간데이터_4,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도
0,2020-01-01,고색동,POINT (126.976384 37.252329),22.600000,21.056205,0.2,2.4,90.0,-6.3,0.2,64.0
1,2020-01-02,고색동,POINT (126.976384 37.252329),49.615385,40.924386,3.2,1.4,92.0,-0.3,0.1,76.0
2,2020-01-03,고색동,POINT (126.976384 37.252329),51.050000,50.600915,5.2,3.0,99.0,-3.9,0.0,51.0
3,2020-01-04,고색동,POINT (126.976384 37.252329),43.375000,41.091517,7.4,3.4,100.0,-4.3,0.1,48.0
4,2020-01-05,고색동,POINT (126.976384 37.252329),36.950000,36.200925,6.2,1.7,96.0,-5.5,0.3,43.0
...,...,...,...,...,...,...,...,...,...,...,...
2707,2020-12-27,호매실동,POINT (126.957008 37.266153),43.583333,44.883084,8.9,2.9,93.0,0.4,0.0,55.0
2708,2020-12-28,호매실동,POINT (126.957008 37.266153),53.041667,49.511207,10.8,2.4,98.0,1.8,0.0,51.0
2709,2020-12-29,호매실동,POINT (126.957008 37.266153),41.826087,44.185211,3.9,5.0,100.0,-4.5,0.1,56.0
2710,2020-12-30,호매실동,POINT (126.957008 37.266153),11.136364,10.812588,-5.4,6.4,68.0,-12.1,2.4,44.0


In [ ]:
df_측정소_보간데이터2.to_csv('df_측정소_보간데이터2.csv', index=False)

## c) 측정소별 가장 가까운 도로 선택 + 그 도로의 교통량 정보 병합  
* 교통량의 경우 12시, 17시 교통량 사용

In [ ]:
df_측정소_도로 = df_측정소[['측정소','geo_cent']]
df_측정소_도로['link_id'] = 0
df_측정소_도로['road_distance'] = float(0)

for i in range(len(df_측정소)):
    a = []
    for j in range(len(df_도로위치)):
        a.append(df_측정소['geo_cent'][i].distance(df_도로위치['geometry'][j]))    
    df_측정소_도로['road_distance'][i] = min(a)
    df_측정소_도로['link_id'][i] = df_도로위치['link_id'][np.argmin(a)]

df_측정소_도로 = pd.merge(left=df_측정소_도로, right=df_교통량_일평균_1217, how='left', on=['link_id'], sort=False)
df_측정소_도로.rename(columns={12:'교통량_12',17:'교통량_17'}, inplace=True)
df_22['lanes'] = pd.to_numeric(df_22['up_lanes']) + pd.to_numeric(df_22['dw_lanes'])
df_측정소_도로 = pd.merge(df_측정소_도로, df_22[['link_id','lanes']], on='link_id', how='left')
df_측정소_도로

,측정소,geo_cent,link_id,road_distance,교통량_12,교통량_17,lanes
0,고색동,POINT (126.976384 37.252329),478341887,0.000618,149.0,205.0,2
1,광교동,POINT (127.070615 37.291321),571474848,0.000626,80.0,97.5,2
2,동수원,POINT (127.030494 37.276744),571461985,0.000007,1200.0,1535.5,4
3,신풍동,POINT (127.010465 37.28385),571410544,0.001081,269.5,373.5,2
4,영통동,POINT (127.056294 37.246901),571341691,0.000086,93.5,123.5,2
5,인계동,POINT (127.028723 37.263637),571440765,0.000803,125.5,167.5,2
6,천천동,POINT (126.975305 37.292877),478446724,0.000882,110.0,154.5,4
7,호매실동,POINT (126.957008 37.266153),478459260,0.000172,648.0,854.5,6


In [ ]:
df_측정소_도로.to_csv("df_측정소_도로.csv", index=False)

## d) 반경 1000m 내 유해시설  
: 분진을 발생시킬 수 있는 주차장 및 공장을 유해시설로 간주.
* 주차장 정보 데이터 : `12.수원시_주차장현황.csv`

In [ ]:
# 공장 개수 변수 생성

df_측정소_유해시설 = pd.concat([df_측정소['측정소'],pd.Series(buffer_1000측정소)],axis=1)
df_측정소_유해시설.columns=['측정소','geometry_factory']
df_측정소_유해시설['factory_num']=0

for j in range(len(df_측정소_유해시설)):
    for i in range(len(factory)):
        if df_측정소_유해시설.geometry_factory[j].contains(factory.geo_cent[i]):
             df_측정소_유해시설['factory_num'][j]+=1
df_측정소_유해시설

,측정소,geometry_factory,factory_num
0,고색동,"POLYGON ((126.98766 37.25238, 126.98761 37.251...",62
1,광교동,"POLYGON ((127.08190 37.29136, 127.08185 37.290...",0
2,동수원,"POLYGON ((127.04177 37.27679, 127.04172 37.275...",0
3,신풍동,"POLYGON ((127.02175 37.28390, 127.02170 37.283...",1
4,영통동,"POLYGON ((127.06757 37.24694, 127.06752 37.246...",201
5,인계동,"POLYGON ((127.04000 37.26368, 127.03995 37.262...",1
6,천천동,"POLYGON ((126.98659 37.29293, 126.98654 37.292...",0
7,호매실동,"POLYGON ((126.96829 37.26620, 126.96824 37.265...",0


In [ ]:
# 주차장여부 변수 생성

df_12['geo_cent'] = [Point([df_12['lon'][i], df_12['lat'][i]]) for i in range(len(df_12))]
df_12 = df_12[df_12['주차구획수']>=100].reset_index(drop=True)

df_측정소_유해시설['주차장여부']=0

for i in range(len(df_측정소_유해시설)):
    for j in range(len(df_12)):
        if buffer_1000측정소[i].contains(df_12['geo_cent'][j]):
            df_측정소_유해시설['주차장여부'][i]=1

# 공장 개수 변수 생성

df_측정소_유해시설 = pd.concat([df_측정소['측정소'],pd.Series(buffer_1000측정소)],axis=1)
df_측정소_유해시설.columns=['측정소','geometry_factory']
df_측정소_유해시설['factory_num']=0

for j in range(len(df_측정소_유해시설)):
    for i in range(len(factory)):
        if df_측정소_유해시설.geometry_factory[j].contains(factory.geo_cent[i]):
             df_측정소_유해시설['factory_num'][j]+=1

df_측정소_유해시설

,측정소,geometry_factory,factory_num
0,고색동,"POLYGON ((126.98766 37.25238, 126.98761 37.251...",62
1,광교동,"POLYGON ((127.08190 37.29136, 127.08185 37.290...",0
2,동수원,"POLYGON ((127.04177 37.27679, 127.04172 37.275...",0
3,신풍동,"POLYGON ((127.02175 37.28390, 127.02170 37.283...",1
4,영통동,"POLYGON ((127.06757 37.24694, 127.06752 37.246...",201
5,인계동,"POLYGON ((127.04000 37.26368, 127.03995 37.262...",1
6,천천동,"POLYGON ((126.98659 37.29293, 126.98654 37.292...",0
7,호매실동,"POLYGON ((126.96829 37.26620, 126.96824 37.265...",0


In [ ]:
df_측정소_유해시설.to_csv("df_측정소_유해시설.csv", index=False)

## e) 반경 500m 내 유동인구  
* 시간대별 유동인구 데이터 : `14.수원시_시간대별_유동인구(2020).csv`

In [ ]:
df_14['geo_cent'] = df_14.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
df_14['측정월']=df_14['STD_YM']%100
df_측정소_유동인구=df_14[['STD_YM','TMST_10','TMST_18','측정월','geo_cent']]
df_측정소_유동인구['측정소'] = 0
for i in range(len(df_측정소_유동인구)):
    for j in range(len(buffer_500측정소)):
            if (buffer_500측정소[j].contains(df_측정소_유동인구.geo_cent[i])):
                df_측정소_유동인구['측정소'][i] = df_측정소['측정소'][j]

df_측정소_유동인구= df_측정소_유동인구.groupby(['측정월','측정소']).sum().reset_index(drop=True)
df_측정소_유동인구= pd.DataFrame.reset_index(df_측정소_유동인구[df_측정소_유동인구['측정소']!=0].drop(['STD_YM', 'index'],axis=1))
df_측정소_유동인구

,측정월,측정소,TMST_10,TMST_18
0,1,고색동,1330.00,1967.57
1,1,광교동,4595.93,4828.28
2,1,동수원,3654.18,4148.58
3,1,신풍동,1299.99,1794.80
4,1,영통동,3657.21,6554.32
...,...,...,...,...
91,12,신풍동,1640.04,2368.56
92,12,영통동,3603.01,6129.35
93,12,인계동,6543.51,9196.74
94,12,천천동,690.18,962.19


In [ ]:
df_측정소_유동인구.to_csv("df_측정소_유동인구.csv", index=False)

# 2) 버스정류장별 데이터셋  
이용시간 예측 모델을 훈련시키기 위한 데이터셋  

## a) 승하차 내역  
* 승하차이력 데이터셋 : `2.수원시_버스정류장별_승하차이력(1) - 6.수원시_버스정류장별_승하차이력(5).csv`

In [ ]:
## df_33 = 버스 승하차 이력 전체 데이터 
df_33=pd.concat([df_02, df_03, df_04, df_05, df_06], axis=0)

## df_34 = 버스 승하차 건수 하루 평균 100 건 이하, BIS 미설치 된 버스정류장 filter
df_33['전체 승하차 건수']=df_33['전체 승차 건수']+df_33['전체 하차 건수']
df_34=df_33.groupby(['정류소ID','일자']).sum()
df_34=df_34.groupby(['정류소ID']).mean()
df_34=pd.DataFrame.reset_index(df_34[df_34['전체 승하차 건수']>100])
df_01=pd.DataFrame.reset_index(df_01[df_01['BIS설치여부']==1])
df_34=pd.merge(df_01[['정류소ID','lon','lat']], df_34, on='정류소ID', how='inner')


df_34['geo_cent'] = [Point([df_34['lon'][i], df_34['lat'][i]]) for i in range(len(df_34))]
df_버스정류장=df_34[['정류소ID','lon','lat','geo_cent']]
df_버스정류장

,정류소ID,lon,lat,geo_cent
0,201000055,127.029815,37.256755,POINT (127.029815 37.25675503)
1,201000268,127.034254,37.246081,POINT (127.034254 37.24608082)
2,201000337,126.941563,37.267677,POINT (126.9415628 37.2676768)
3,202000089,126.990494,37.284140,POINT (126.9904943 37.28414003)
4,201000358,126.952716,37.265969,POINT (126.9527161 37.2659687)
...,...,...,...,...
693,202000233,126.996651,37.281358,POINT (126.9966511 37.28135833)
694,202000232,126.999037,37.279071,POINT (126.9990373 37.27907114)
695,202000249,126.999697,37.278865,POINT (126.9996969 37.27886499)
696,202000241,126.995823,37.284316,POINT (126.9958225 37.28431556)


In [ ]:
## 정류소 반경 500m buffer 
buffer_500= gpd.GeoDataFrame(df_34['geo_cent'], geometry='geo_cent')
buffer_500.crs = {'init':'epsg:4326'} # 좌표계가 지정되지 않아 알맞는 4326 좌표계를 설정해줍니다.
buffer_500= buffer_500.to_crs({'init':'epsg:5179'}) # 4326 좌표계는 미터단위 분석이 불가능하므로 100미터 버퍼 생성이 가능한 5179 좌표계로 변환합니다.
buffer_500=buffer_500.buffer(500)
buffer_500=buffer_500.to_crs({'init':'epsg:4326'}) # 버퍼 후 4326 좌표계로 되돌아옵니다.

In [ ]:
## 정류소 반경 1000m buffer 
buffer_1000= gpd.GeoDataFrame(df_34['geo_cent'], geometry='geo_cent')
buffer_1000.crs = {'init':'epsg:4326'} # 좌표계가 지정되지 않아 알맞는 4326 좌표계를 설정해줍니다.
buffer_1000= buffer_1000.to_crs({'init':'epsg:5179'}) # 4326 좌표계는 미터단위 분석이 불가능하므로 100미터 버퍼 생성이 가능한 5179 좌표계로 변환합니다.
buffer_1000=buffer_1000.buffer(1000)
buffer_1000=buffer_1000.to_crs({'init':'epsg:4326'}) # 버퍼 후 4326 좌표계로 되돌아옵니다.

## b) 인구정보  
버스정류장 반경 500m 내에 포함되는 인구 정보를 모두 더해 버스정류장별 인구데이터로 변환  
* 인구 데이터 : `17.수원시_인구정보(고령)_격자.geojson`, `18.수원시_인구정보(생산가능)_격자.geojson`, `19.수원시_인구정보(유소년)_격자.geojson'`

In [ ]:
df_인구 = df_17.drop(['val'], axis=1)
df_인구 = df_인구.merge(df_17.drop(['geometry'], axis=1), how='inner', on='gid')
df_인구 = df_인구.merge(df_18.drop(['geometry'], axis=1), how='inner', on='gid')
df_인구 = df_인구.merge(df_19.drop(['geometry'], axis=1), how='inner', on='gid')
df_인구 = df_인구.fillna(0)
df_인구['인구수_전체'] = df_인구.iloc[:,2] + df_인구.iloc[:,3] + df_인구.iloc[:,4]
df_인구.columns = ['gid', 'geometry', '인구수_고령', '인구수_생산가능', '인구수_유소년', '인구수_전체']

In [ ]:
df_인구['coord_cent'] = df_인구['geometry'].apply(lambda x : list(x[0].centroid.coords[0]))
df_인구['geo_cent'] = df_인구['coord_cent'].apply(lambda x : Point(x))

In [ ]:
df_버스_인구=df_버스정류장[['정류소ID']]
df_버스_인구['인구수_유소년'] = 0
df_버스_인구['인구수_생산가능'] = 0
df_버스_인구['인구수_고령'] = 0

for i in range(len(df_인구)):
    for j in range(len(df_버스정류장)):
        if buffer_500[j].contains(df_인구.geo_cent[i]):
            df_버스_인구['인구수_유소년'][j]+=df_인구.인구수_유소년[i]
            df_버스_인구['인구수_생산가능'][j]+=df_인구.인구수_생산가능[i]
            df_버스_인구['인구수_고령'][j]+=df_인구.인구수_고령[i]

df_버스_인구

,정류소ID,인구수_유소년,인구수_생산가능,인구수_고령
0,201000055,1457,12643,1711
1,201000268,1996,14255,1327
2,201000337,1491,6175,1046
3,202000089,1067,8335,1482
4,201000358,2230,10242,1394
...,...,...,...,...
693,202000233,2177,14797,3293
694,202000232,1853,13785,3422
695,202000249,1698,12519,3093
696,202000241,1897,12798,2426


In [ ]:
df_버스_인구.to_csv("df_버스_인구.csv", index=False)

## c) 버스 노선 개수  
* 버스정류장별 노선현황 데이터 : `7.수원시_버스정류장별_노선현황.csv`

In [ ]:
df_07['노선개수']=df_07['운행노선'].map(lambda x : x.count(',')+1)
df_07['정류소ID']=df_07['정류소ID'].astype(int)
df_버스_노선개수=df_07[['정류소ID','노선개수']]
df_버스_노선개수

,정류소ID,노선개수
0,200000001,1
1,200000002,1
2,200000003,1
3,200000004,1
4,200000005,1
...,...,...
1123,233000002,10
1124,233000592,10
1125,233000593,10
1126,233001574,8


In [ ]:
df_버스_노선개수.to_csv("df_버스_노선개수.csv", index=False)

## d) 버스정류장별 가장 가까운 도로 선택 + 해당 도로의 정보 병합

### d-1) 버스정류장별 가장 가까운 도로 선택 + 해당 도로와의 거리 변수 생성

In [ ]:
df_버스_도로 = df_버스정류장[['정류소ID']]
df_버스_도로['link_id'] = 0
df_버스_도로['road_distance'] = float(0)
for i in range(len(df_버스_도로)):
    a = []
    for j in range(len(df_도로위치)):
        a.append(df_버스정류장['geo_cent'][i].distance(df_도로위치['geometry'][j]))    
    df_버스_도로['road_distance'][i] = min(a)
    df_버스_도로['link_id'][i] = df_도로위치['link_id'][np.argmin(a)]
df_버스_도로

,정류소ID,link_id,road_distance
0,201000055,571410962,0.000088
1,201000268,571369580,0.000229
2,201000337,478458799,0.000087
3,202000089,478451157,0.000240
4,201000358,478444829,0.000068
...,...,...,...
693,202000233,478410636,0.000052
694,202000232,571410708,0.000050
695,202000249,571410304,0.000037
696,202000241,478450414,0.000101


### d-2) 도로 정보(교통량, 차선수, 폭, 혼잡지표) 추가  
- 교통량의 경우 12시, 17시 교통량 사용

In [ ]:
df_22['lanes'] = pd.to_numeric(df_22['up_lanes']) + pd.to_numeric(df_22['dw_lanes'])
df_22_ = df_22[['link_id','lanes','width']]
df_22_['link_id'] = pd.to_numeric(df_22_['link_id'])
df_24['link_id'] = round(df_24['상세도로망_LinkID']/100)
df_25['link_id'] = round(df_25['상세도로망_LinkID']/100)
df_버스_도로 = pd.merge(df_버스_도로,df_22_, how='left', on='link_id')
df_버스_도로 = df_버스_도로.merge(df_24.groupby(['link_id']).mean().reset_index().drop(['상세도로망_LinkID', '도로등급', '링크길이'], axis=1),how='left', on='link_id')
df_버스_도로 = df_버스_도로.merge(df_25.groupby(['link_id']).mean().reset_index().drop(['상세도로망_LinkID', '도로등급', '링크길이'], axis=1), how='left', on='link_id')
df_버스_도로 = pd.merge(left=df_버스_도로, right=df_교통량_일평균_1217, how='left', on=['link_id'], sort=False)
df_버스_도로

,정류소ID,link_id,road_distance,lanes,width,혼잡빈도강도,혼잡시간강도,12,17
0,201000055,571410962,0.000088,4,3,50.400,82.260,477.0,607.5
1,201000268,571369580,0.000229,10,5,30.810,48.760,2131.5,2956.0
2,201000337,478458799,0.000087,4,3,16.820,42.515,277.5,364.0
3,202000089,478451157,0.000240,8,4,17.305,42.075,1671.0,2139.0
4,201000358,478444829,0.000068,6,4,9.930,32.255,588.5,761.5
...,...,...,...,...,...,...,...,...,...
693,202000233,478410636,0.000052,2,2,47.255,75.405,332.5,467.5
694,202000232,571410708,0.000050,2,2,55.670,75.625,322.5,451.5
695,202000249,571410304,0.000037,2,2,74.065,87.915,365.5,504.5
696,202000241,478450414,0.000101,2,2,32.675,63.200,251.5,356.0


In [ ]:
df_버스_도로.to_csv("df_버스_도로.csv", index=False)

## e) 유동인구
* 요일별 유동인구 데이터 : `16.수원시_요일별_유동인구(2020).csv`  
* 시간대별 유동인구 데이터 : `14.수원시_시간대별_유동인구(2020).csv`

### e-1) 버스정류장별 요일별 유동인구
버스정류장 반경 500m 내에 포함되는 일별 / 요일별 유동인구 정보를 모두 더해 버스정류장별 유동인구 데이터로 변환  




In [ ]:
df_버스_유동인구 = df_버스정류장[['정류소ID']]
요일별_유동인구_격자 = pd.DataFrame.reset_index(df_16.groupby(['lon', 'lat']).sum())

# 유동인구 데이터의 unique한 100x100 격자에 grid_id 부여
요일별_유동인구_격자['grid_id'] = 0
idx = []  
for i in range(len(요일별_유동인구_격자)):
    idx.append(str(i).zfill(5))
요일별_유동인구_격자['grid_id'] = pd.DataFrame(idx) 
요일별_유동인구_격자['geo_cent'] = [Point([요일별_유동인구_격자['lon'][i], 요일별_유동인구_격자['lat'][i]]) for i in range(len(요일별_유동인구_격자))] 

In [ ]:
for i in range(len(요일별_유동인구_격자)):
    df_버스_유동인구[str(i)] = float(0)
    
# 해당 격자의 central point가 반경 500m 에 포함되면 1로 표시
for i in range(len(df_버스_유동인구)):
    for j in range(len(요일별_유동인구_격자)):
        if buffer_500[i].contains(요일별_유동인구_격자['geo_cent'][j]):
            df_버스_유동인구[str(j)][i] = 1  

In [ ]:
df_버스_유동인구.to_csv("df_버스_유동인구_indexing.csv", index=False) # 중간 데이터 저장

In [ ]:
df_버스_유동인구 = pd.read_csv("df_버스_유동인구_indexing.csv") # 중간 데이터 불러오기

In [ ]:
# 요일별 유동인구의 최종데이터 생성을 위한 데이터셋 준비
df_버스_유동인구_요일 = pd.concat([df_버스정류장]*84, ignore_index=True)
df_버스_유동인구_요일['월'] = 0
for i in range(12):
    df_버스_유동인구_요일['월'][i*7*len(df_버스정류장): (i+1)*7*len(df_버스정류장)]=i+1
weekday = [0,1,2,3,4,5,6]*len(df_버스정류장)
df_버스_유동인구_요일['요일'] = pd.concat([pd.DataFrame(weekday)]*84, ignore_index=True)
df_버스_유동인구_요일['유동인구'] = float(0)

요일별_유동인구_위경도 = pd.merge(요일별_유동인구_격자[['lon','lat','grid_id']], df_16, on = ['lon', 'lat'], how = 'left')
요일별_유동인구_위경도['월'] = 요일별_유동인구_위경도['STD_YM']%100

res1 = []
for i in range(len(요일별_유동인구_격자)):
    res1.append(str(i))
idx = pd.DataFrame(res1, columns = ['grid_id'])

# 각 버스정류장 정보에 요일별 유동인구 부여
res2 = []
for i in range(12):
    월 = pd.DataFrame.reset_index(요일별_유동인구_위경도[요일별_유동인구_위경도['월'] == i+1])
    월 = pd.merge(idx, 월, on = 'grid_id', how = 'left')
    월 = 월.fillna(0)
    for j in range(len(df_버스_유동인구)):
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,5]))
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,6]))
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,7]))
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,8]))
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,9]))
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,10]))
        res2.append(np.array(df_버스_유동인구.iloc[j,1:]).dot(월.iloc[:,11]))        
df_버스_유동인구_요일['유동인구'] = res2
df_버스_유동인구_요일

,정류소ID,lon,lat,geo_cent,월,요일,유동인구
0,201000055,127.029815,37.256755,POINT (127.029815 37.25675503),1,0,51131.54
1,201000268,127.034254,37.246081,POINT (127.034254 37.24608082),1,1,54919.17
2,201000337,126.941563,37.267677,POINT (126.9415628 37.2676768),1,2,52582.93
3,202000089,126.990494,37.284140,POINT (126.9904943 37.28414003),1,3,55553.55
4,201000358,126.952716,37.265969,POINT (126.9527161 37.2659687),1,4,54580.57
...,...,...,...,...,...,...,...
58627,202000233,126.996651,37.281358,POINT (126.9966511 37.28135833),12,2,40101.59
58628,202000232,126.999037,37.279071,POINT (126.9990373 37.27907114),12,3,41023.17
58629,202000249,126.999697,37.278865,POINT (126.9996969 37.27886499),12,4,41118.53
58630,202000241,126.995823,37.284316,POINT (126.9958225 37.28431556),12,5,39470.66


In [ ]:
df_버스_유동인구_요일.to_csv("df_버스_유동인구_요일.csv", index=False)

### e-2) 버스정류장별 시간대별 유동인구
버스정류장 반경 500m 내에 포함되는 시간대별 유동인구 정보를 모두 더해 버스정류장별 유동인구 데이터로 변환  


In [ ]:
시간대별_유동인구_격자 = pd.DataFrame.reset_index(df_14.groupby(['lon','lat']).sum())
시간대별_유동인구_격자['grid_id'] = 0
idx = []
for i in range(len(시간대별_유동인구_격자)):
    idx.append(str(i).zfill(5))
시간대별_유동인구_격자['grid_id'] = pd.DataFrame(idx)
시간대별_유동인구_격자['geo_cent'] = 시간대별_유동인구_격자.apply(lambda row : Point([row['lon'], row['lat']]), axis = 1)

In [ ]:
df_버스_유동인구 = df_버스정류장[['정류소ID']]
for i in range(len(시간대별_유동인구_격자)):
    df_버스_유동인구[str(i)] = float(0)

for i in range(len(df_버스_유동인구)):
    for j in range(len(시간대별_유동인구_격자)):
        if buffer_500[i].contains(시간대별_유동인구_격자['geo_cent'][j]):
            df_버스_유동인구[str(j)][i] = 1

In [ ]:
df_버스_유동인구.to_csv("df_버스_유동인구2_indexing.csv", index=False) # 중간 데이터 저장

In [ ]:
df_버스_유동인구 = pd.read_csv("df_버스_유동인구2_indexing.csv") # 중간 데이터 불러오기

In [ ]:
# 시간대별 유동인구의 최종데이터 생성을 위한 데이터셋 준비
df_버스_유동인구_시간 = df_버스정류장[['정류소ID']]
df_버스_유동인구_시간 = pd.concat([df_버스_유동인구_시간]*12).reset_index(drop=True)
df_버스_유동인구_시간['월'] = [1]*len(df_버스정류장)+[2]*len(df_버스정류장)+[3]*len(df_버스정류장)+[4]*len(df_버스정류장)+[5]*len(df_버스정류장)+[6]*len(df_버스정류장)+[7]*len(df_버스정류장)+[8]*len(df_버스정류장)+[9]*len(df_버스정류장)+[10]*len(df_버스정류장)+[11]*len(df_버스정류장)+[12]*len(df_버스정류장)
df_버스_유동인구_시간['유동인구_10'] = float(0)
df_버스_유동인구_시간['유동인구_18'] = float(0)

시간대별_유동인구_위경도 = pd.merge(df_14, 시간대별_유동인구_격자[['grid_id','lon','lat']], on = ['lon','lat'], how = 'right')
시간대별_유동인구_위경도['월'] = 시간대별_유동인구_위경도['STD_YM']%100

res1 = []
for i in range(len(시간대별_유동인구_격자)):
    res1.append(str(i))
idx = pd.DataFrame(res1, columns = ['grid_id'])

# 각 버스정류장 정보에 시간대별 유동인구 부여
res2 = []
res3 = []
for i in range(12):
    월 = 시간대별_유동인구_위경도[시간대별_유동인구_위경도['월'] == (i+1)].reset_index(drop=True)
    월 = pd.merge(idx, 월, on = ['grid_id'], how = 'left').fillna(0)[['grid_id','TMST_10','TMST_18']]
    for k in range(len(df_버스정류장)):
        res2.append(np.array(월.iloc[:, 1]).dot(df_버스_유동인구.iloc[k,1:]))
        res3.append(np.array(월.iloc[:, 2]).dot(df_버스_유동인구.iloc[k,1:]))
df_버스_유동인구_시간['유동인구_10'] = res2        
df_버스_유동인구_시간['유동인구_18'] = res3
df_버스_유동인구_시간

,정류소ID,월,유동인구_10,유동인구_18
0,201000055,1,2692.44,3547.08
1,201000268,1,4376.12,6178.56
2,201000337,1,0.00,0.00
3,202000089,1,3021.93,4869.78
4,201000358,1,0.00,0.00
...,...,...,...,...
8371,202000233,12,1606.74,2246.08
8372,202000232,12,1767.75,2400.69
8373,202000249,12,1698.97,2316.22
8374,202000241,12,1800.23,2769.06


In [ ]:
df_버스_유동인구_시간.to_csv("df_버스_유동인구_시간.csv", index=False)

## f) 미세먼지/초미세먼지 측정량을 이용해 버스정류장별 보간데이터 생성 후, 날씨 데이터와 병합
* 대기환경 데이터 : 2020년 수원시 시간별 대기질 측정정보 / `2020년수원미세먼지.csv` /  출처 : [경기데이터드림](https://data.gg.go.kr/portal/data/village/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=GE0DUHTX3VX0GL4R0LUS26448884&infSeq=1&order=&sigunNm=%EC%88%98%EC%9B%90%EC%8B%9C&sigunFlag=41110&searchWord=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80)  
* 날씨 데이터 : `13.수원시_기상데이터(2020).csv`

### f-1) 미세먼지

In [ ]:
# ===== 버스정류장별 미세먼지를 예측하기 위한 예측 데이터(X_test) 생성 =====

df_버스 = pd.merge(df_버스정류장, df_34[['정류소ID', '전체 승하차 건수']], on = '정류소ID', how = 'inner')
df_버스[['고색동', '광교동', '신풍동', '영통동', '인계동', '천천동', '호매실동', '동수원']] = 0

# 버스정류장과 측정소간 거리 계산
for i in range(len(df_버스)):
    for j in range(len(df_측정소)):
        df_버스.iloc[i,j+5] = df_버스['geo_cent'][i].distance(df_측정소['geo_cent'][j])

# 버스정류장별 보간데이터 계산을 위한 데이터셋 생성
df_미세_ = pd.DataFrame.reset_index(pd.concat([df_미세]*len(df_버스))).drop(['index'], axis = 1)
df_미세_['정류소ID'] = 0
for i in range(len(df_버스)):
    df_미세_['정류소ID'][i*len(df_미세):(i+1)*len(df_미세)] = df_버스['정류소ID'][i]
df_정류소 = pd.merge(df_미세_, df_버스, on = '정류소ID', how = "left")    
df_버스_보간데이터 = df_정류소[['날짜','정류소ID']]
df_버스_보간데이터['target'] = 0
df_버스_보간데이터['보간데이터'] = float(0)

# 버스정류장별 보간데이터 입력
for i in range(len(df_정류소)):
    order = np.array(df_정류소.iloc[i,14:]).argsort().tolist()
    b_5 = sum(1/np.array(df_정류소.iloc[i,14:])[order][1:5])
    a_5 = sum(1/np.array(df_정류소.iloc[i,14:])[order][1:5] * np.array(df_정류소.iloc[i,1:9])[order][1:5])
    df_버스_보간데이터['보간데이터'][i] = float(a_5 / b_5)

# 날씨 데이터 병합
df_버스_보간데이터 = pd.merge(df_버스_보간데이터, df_max[['날짜','최고기온','최고풍속', '최고습도']], on='날짜', how="left")
df_버스_보간데이터 = pd.merge(df_버스_보간데이터, df_min[['날짜','최저기온','최저풍속', '최저습도']], on='날짜', how="left")
df_버스_보간데이터 = pd.DataFrame.reset_index(df_버스_보간데이터.dropna()).drop(['index'], axis=1)  
df_버스_보간데이터

,날짜,정류소ID,target,보간데이터,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도
0,2020-01-01,201000055,0,37.005345,0.2,2.4,90.0,-6.3,0.2,64.0
1,2020-01-02,201000055,0,68.316946,3.2,1.4,92.0,-0.3,0.1,76.0
2,2020-01-03,201000055,0,80.674983,5.2,3.0,99.0,-3.9,0.0,51.0
3,2020-01-04,201000055,0,65.811928,7.4,3.4,100.0,-4.3,0.1,48.0
4,2020-01-05,201000055,0,57.449223,6.2,1.7,96.0,-5.5,0.3,43.0
...,...,...,...,...,...,...,...,...,...,...
243083,2020-12-27,202000246,0,71.631779,8.9,2.9,93.0,0.4,0.0,55.0
243084,2020-12-28,202000246,0,73.654294,10.8,2.4,98.0,1.8,0.0,51.0
243085,2020-12-29,202000246,0,68.459205,3.9,5.0,100.0,-4.5,0.1,56.0
243086,2020-12-30,202000246,0,32.700749,-5.4,6.4,68.0,-12.1,2.4,44.0


In [ ]:
df_버스_보간데이터.to_csv("df_버스_보간데이터.csv", index=False)

### f-2) 초미세먼지

In [ ]:
# ===== 버스정류장별 초미세먼지를 예측하기 위한 예측 데이터(X_test) 생성 =====

df_버스 = pd.merge(df_버스정류장, df_34[['정류소ID', '전체 승하차 건수']], on = '정류소ID', how = 'inner')
df_버스[['고색동', '광교동', '신풍동', '영통동', '인계동', '천천동', '호매실동', '동수원']] = 0

# 버스정류장과 측정소간 거리 계산
for i in range(len(df_버스)):
    for j in range(len(df_측정소)):
        df_버스.iloc[i,j+5] = df_버스['geo_cent'][i].distance(df_측정소['geo_cent'][j])

# 버스정류장별 보간데이터 계산을 위한 데이터셋 생성
df_초미세_ = pd.DataFrame.reset_index(pd.concat([df_초미세]*len(df_버스))).drop(['index'], axis = 1)
df_초미세_['정류소ID'] = 0
for i in range(len(df_버스)):
    df_초미세_['정류소ID'][i*len(df_초미세):(i+1)*len(df_초미세)] = df_버스['정류소ID'][i]
df_정류소 = pd.merge(df_초미세_, df_버스, on='정류소ID', how="left")    

df_버스_보간데이터2 = df_정류소[['날짜','정류소ID']]
df_버스_보간데이터2['target'] = 0
df_버스_보간데이터2['보간데이터'] = float(0)

# 버스정류장별 보간데이터 입력
for i in range(len(df_정류소)):
    order = np.array(df_정류소.iloc[i,14:]).argsort().tolist()
    b_5 = sum(1/np.array(df_정류소.iloc[i,14:])[order][1:5])
    a_5 = sum(1/np.array(df_정류소.iloc[i,14:])[order][1:5] * np.array(df_정류소.iloc[i,1:9])[order][1:5])
    df_버스_보간데이터2['보간데이터'][i] = float(a_5 / b_5)
    
# 날씨 데이터 병합
df_버스_보간데이터2 = pd.merge(df_버스_보간데이터2, df_max[['날짜','최고기온','최고풍속', '최고습도']], on = '날짜', how = "left")
df_버스_보간데이터2 = pd.merge(df_버스_보간데이터2, df_min[['날짜','최저기온','최저풍속', '최저습도']], on = '날짜', how = "left")
df_버스_보간데이터2 = pd.DataFrame.reset_index(df_버스_보간데이터2.dropna()).drop(['index'], axis = 1)  
df_버스_보간데이터2

,날짜,정류소ID,target,보간데이터,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도
0,2020-01-01,201000055,0,22.837865,0.2,2.4,90.0,-6.3,0.2,64.0
1,2020-01-02,201000055,0,43.541571,3.2,1.4,92.0,-0.3,0.1,76.0
2,2020-01-03,201000055,0,53.757842,5.2,3.0,99.0,-3.9,0.0,51.0
3,2020-01-04,201000055,0,44.082389,7.4,3.4,100.0,-4.3,0.1,48.0
4,2020-01-05,201000055,0,37.471504,6.2,1.7,96.0,-5.5,0.3,43.0
...,...,...,...,...,...,...,...,...,...,...
243117,2020-12-27,202000246,0,47.033909,8.9,2.9,93.0,0.4,0.0,55.0
243118,2020-12-28,202000246,0,55.339784,10.8,2.4,98.0,1.8,0.0,51.0
243119,2020-12-29,202000246,0,45.330737,3.9,5.0,100.0,-4.5,0.1,56.0
243120,2020-12-30,202000246,0,9.633114,-5.4,6.4,68.0,-12.1,2.4,44.0


In [ ]:
df_버스_보간데이터2.to_csv("df_버스_보간데이터2.csv", index=False)

## g) 반경 1000m 내 유해시설  
: 분진을 발생시킬 수 있는 주차장 및 공장을 유해시설로 간주.
* 주차장 정보 데이터 : `12.수원시_주차장현황.csv`

In [ ]:
# 공장 개수 변수 생성

df_버스_유해시설 = pd.concat([df_버스정류장['정류소ID'].drop_duplicates().reset_index(drop=True),pd.Series(buffer_1000)],axis = 1)
df_버스_유해시설.columns = ['정류소ID','geometry_factory']
df_버스_유해시설['factory_num'] = 0

for j in range(len(df_버스_유해시설)):
    for i in range(len(factory)):
        if df_버스_유해시설.geometry_factory[j].contains(factory.geo_cent[i]):
             df_버스_유해시설['factory_num'][j] += 1

In [ ]:
# 주차장 존재 여부 변수 생성

df_12['geo_cent'] = [Point([df_12['lon'][i], df_12['lat'][i]]) for i in range(len(df_12))]
df_12 = df_12[df_12['주차구획수'] >= 100].reset_index(drop=True)

df_버스_유해시설['주차장여부'] = 0

for i in range(len(df_버스_유해시설)):
    for j in range(len(df_12)):
        if buffer_1000[i].contains(df_12['geo_cent'][j]):
            df_버스_유해시설['주차장여부'][i] = 1
df_버스_유해시설

,정류소ID,geometry_factory,factory_num,주차장여부
0,201000055,"POLYGON ((127.04109 37.25680, 127.04104 37.255...",1,1
1,201000268,"POLYGON ((127.04553 37.24612, 127.04548 37.245...",4,1
2,201000337,"POLYGON ((126.95284 37.26773, 126.95279 37.266...",0,0
3,202000089,"POLYGON ((127.00177 37.28419, 127.00173 37.283...",0,1
4,201000358,"POLYGON ((126.96399 37.26602, 126.96395 37.265...",0,0
...,...,...,...,...
693,202000233,"POLYGON ((127.00793 37.28141, 127.00788 37.280...",2,1
694,202000232,"POLYGON ((127.01032 37.27912, 127.01027 37.278...",3,0
695,202000249,"POLYGON ((127.01098 37.27891, 127.01093 37.278...",3,0
696,202000241,"POLYGON ((127.00710 37.28436, 127.00705 37.283...",0,1


In [ ]:
df_버스_유해시설.to_csv("df_버스_유해시설.csv", index=False)

## h)  반경 500m 내 주요 시설과의 거리 평균
: 집, 회사, 학교, 상업시설, 관공서, 공장

In [ ]:
# 버스정류장 500m 반경 내에 위치한 건물들과 버스정류장 사이의 거리의 평균을 구하는 함수

def building_distance(data):
    mean_val = pd.Series(np.repeat(float(0),len(df_버스정류장)))
    for j in range(len(df_버스정류장)):
        a = []
        for i in range(len(data)):
            if buffer_500[j].contains(data.geo_cent[i]):
                a.append(df_버스정류장.geo_cent[j].distance(data.geo_cent[i]))  
                mean_val[j] = sum(a)/len(a)
    return pd.Series(mean_val)

# 버스정류장 500m 반경 내에 위치한 건물의 개수를 구하는 함수

def building_nums(data):
    val = pd.Series(np.repeat(float(0),len(df_버스정류장)))
    for j in range(len(df_버스정류장)):
        for i in range(len(data)):
            if buffer_500[j].contains(data.geo_cent[i]):
                val[j] += 1 
    return pd.Series(val)   

In [ ]:
# 버스정류장 반경 500m에 있는 학교, 주택, 공장 등의 개수 및 유무 반영 코드

df_버스_건물 = df_버스정류장[['정류소ID']]
df_버스_건물[['학교_5분이내거리평균']] = building_distance(school)
df_버스_건물[['집_5분이내거리평균']] = building_distance(house)
df_버스_건물[['회사_5분이내거리평균']] = building_distance(company)
df_버스_건물[['공장_5분이내거리평균']] = building_distance(factory)
df_버스_건물[['상업시설_5분이내거리평균']] = building_distance(store)
df_버스_건물[['관공서_5분이내거리평균']] = building_distance(public)
df_버스_건물[['병원_5분이내거리평균']] = building_distance(hospi)
df_버스_건물[['지하철_5분이내거리평균']] = building_distance(df_08)

df_버스_건물['학교_5분이내개수'] = building_nums(school)
df_버스_건물['집_5분이내개수'] = building_nums(house)
df_버스_건물['회사_5분이내개수'] = building_nums(company)
df_버스_건물['공장_5분이내개수'] = building_nums(factory)
df_버스_건물['상업시설_5분이내개수'] = building_nums(store)
df_버스_건물['관공서_5분이내개수'] = building_nums(public)
df_버스_건물['병원_5분이내개수'] = building_nums(public)
df_버스_건물['지하철_5분이내개수'] = building_nums(df_08)

In [ ]:
df_버스_건물.to_csv("df_버스_건물.csv", index = False)

## j) 정류장 유형  
* 버스정류장 정보 데이터 : `1.수원시_버스정류장.csv`

In [ ]:
# 정류소유형 파생변수
df_버스_유형 = pd.merge(df_01, df_버스정류장, on = '정류소ID', how = 'right')[['정류소ID','정류장유형']]
df_버스_유형['시외여부'] = list(1 if '시외' in df_버스_유형['정류장유형'][i] else 0 for i in range(len(df_버스정류장)))
df_버스_유형['마을여부'] = list(1 if '마을' in df_버스_유형['정류장유형'][i] else 0 for i in range(len(df_버스정류장)))
df_버스_유형['공항여부'] = list(1 if '공항' in df_버스_유형['정류장유형'][i] else 0 for i in range(len(df_버스정류장)))

df_버스_유형 

,정류소ID,정류장유형,시외여부,마을여부,공항여부
0,201000055,시내,0,0,0
1,201000268,시내,0,0,0
2,201000337,시내,0,0,0
3,202000089,"시내,마을",0,1,0
4,201000358,시내,0,0,0
...,...,...,...,...,...
693,202000233,"시내,마을",0,1,0
694,202000232,"시내,마을",0,1,0
695,202000249,시내,0,0,0
696,202000241,"시내,마을",0,1,0


In [ ]:
df_버스_유형.to_csv("df_버스_유형.csv", index=False)

## k) 배차시간
* 배차시간 데이터 : 2020년 수원시 버스 배차시간 데이터 / `GGD_RouteInfo_M.csv` / 출처 : [경기버스정보](http://www.gbis.go.kr/)

In [ ]:
# 버스 노선 데이터
df_승하차 = pd.concat([df_02, df_03, df_04, df_05, df_06], axis = 0).drop('미태그 건수', axis = 1)
df_승하차 = df_승하차.drop_duplicates().reset_index()
df_승하차['노선번호'] = df_승하차['노선번호'].apply(lambda x : re.sub('_','-',str(x)))
df_승하차.loc[(df_승하차['노선번호'] == '4000'),'업체명'] = '성우운수' # 4000번 버스의 업체명이 잘못 들어가있음
df_승하차['운행업체'] = df_승하차['업체명'].apply(lambda x : str(x).replace("(주)", ""))
df_승하차['운행업체'] = df_승하차['운행업체'].apply(lambda x : str(x).replace("버스라인", ""))
df_승하차['운행업체'] = df_승하차['운행업체'].replace('경진여객운수','경진여객')
df_승하차 = df_승하차.groupby(['운행업체', '노선번호']).sum().reset_index()[['운행업체','노선번호']]

df_경기버스배차 = pd.read_csv("input/GGD_RouteInfo_M.csv")
df_경기버스배차['운행업체'] = df_경기버스배차['운행업체'].replace('용남버스','용남고속')
df_경기버스배차 = df_경기버스배차[['운행업체', '노선번호','주중배차간격','주말배차간격']]
df_버스노선 = pd.merge(df_승하차,df_경기버스배차, on = ['운행업체', '노선번호'], how='left').drop_duplicates().reset_index()
df_버스노선.iloc[139,3] = df_버스노선.iloc[106,3]
df_버스노선.iloc[139,4] = df_버스노선.iloc[106,4]
df_버스노선.iloc[140,3] = df_버스노선.iloc[107,3]
df_버스노선.iloc[140,4] = df_버스노선.iloc[107,4]
df_버스노선.iloc[127,3] = df_버스노선.iloc[93,3]
df_버스노선.iloc[127,4] = df_버스노선.iloc[93,4]
df_버스노선=df_버스노선.dropna().reset_index()

def 배차_weekend(data):
    a = data['주말배차간격'].astype(str).apply(lambda x: re.sub('분','',x))
    b = a.astype(str).apply(lambda x: x.split('~'))

    data['주말배차'] = float(0)
    for i in range(len(data)):
        if len(b[i]) == 2:
            data['주말배차'][i] = (int(b[i][0])+int(b[i][1]))/2.0
        elif len(b[i]) == 1:
            data['주말배차'][i] = b[i][0]
    return data['주말배차']  
df_버스노선['평일배차'] = 배차_week(df_버스노선)
df_버스노선['주말배차'] = 배차_weekend(df_버스노선)
df_버스_간격 = df_버스노선[['운행업체', '노선번호','평일배차', '주말배차']]
df_버스_간격 = pd.concat([df_버스_간격]*2).reset_index().drop('index',axis=1)
df_버스_간격['주말여부'] = [0]*(148)+[1]*(148)
df_버스_간격['최종배차'] = float(0)

for i in range(len(df_버스_간격)):
    if df_버스_간격['주말여부'][i] == 0:
        df_버스_간격['최종배차'][i] = df_버스_간격['평일배차'][i]/2
    else:
        df_버스_간격['최종배차'][i] = df_버스_간격['주말배차'][i]/2
a = df_버스_간격['최종배차'].apply(lambda x : 0 if x > 30 else x)
df_버스_간격['최종배차'] = a.replace(0, np.mean(a))
df_버스_간격

,운행업체,노선번호,평일배차,주말배차,주말여부,최종배차
0,경기고속,700-2,14.0,20.0,0,7.000000
1,경기고속,720-1,14.0,25.0,0,7.000000
2,경기고속,720-2,13.0,15.0,0,6.500000
3,경기고속,8401,108.0,760.0,0,8.606081
4,경기고속,8409,70.0,70.0,0,8.606081
...,...,...,...,...,...,...
291,화성여객,202,144.0,135.0,1,8.606081
292,화성여객,203,174.0,165.0,1,8.606081
293,화성여객,720-3,28.0,45.0,1,22.500000
294,화성여객,721,58.0,55.0,1,27.500000


In [ ]:
df_버스_간격.to_csv("df_버스_간격.csv", index=False)

# 3) 모델 훈련데이터 / 예측데이터  


## a) 미세먼지 · 초미세먼지 학습데이터

### a-1) 미세먼지

In [ ]:
df_측정소_도로 = pd.read_csv("df_측정소_도로.csv")
df_측정소_보간데이터 = pd.read_csv("df_측정소_보간데이터.csv")
df_측정소_보간데이터['측정월'] = pd.to_datetime(pd.to_datetime(df_측정소_보간데이터['날짜'])).dt.month
df_측정소_유동인구 = pd.read_csv("df_측정소_유동인구.csv")
df_측정소_유해시설 = pd.read_csv("df_측정소_유해시설.csv")
미세먼지_train = pd.merge(df_측정소_보간데이터, df_측정소_도로, on = '측정소', how = 'left').drop(['link_id', 'geo_cent_x','geo_cent_y'], axis = 1)
미세먼지_train = pd.merge(미세먼지_train, df_측정소_유동인구, on = ['측정소','측정월'], how = 'left')
미세먼지_train = pd.merge(미세먼지_train, df_측정소_유해시설, on = '측정소', how = 'left').drop('geometry_factory', axis = 1)

In [ ]:
미세먼지_train.to_csv("미세먼지_train.csv", index=False)

In [ ]:
print(미세먼지_train.shape)
미세먼지_train.head()

(2704, 19)


,날짜,측정소,target,보간데이터_4,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도,측정월,road_distance,교통량_12,교통량_17,lanes,TMST_10,TMST_18,factory_num,주차장여부
0,2020-01-01,고색동,30.173913,32.259131,0.2,2.4,90.0,-6.3,0.2,64.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
1,2020-01-02,고색동,51.869565,58.040780,3.2,1.4,92.0,-0.3,0.1,76.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
2,2020-01-03,고색동,63.652174,69.981241,5.2,3.0,99.0,-3.9,0.0,51.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
3,2020-01-04,고색동,51.260870,56.669166,7.4,3.4,100.0,-4.3,0.1,48.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
4,2020-01-05,고색동,44.416667,50.191075,6.2,1.7,96.0,-5.5,0.3,43.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1


### a-2) 초미세먼지

In [ ]:
df_측정소_도로 = pd.read_csv("df_측정소_도로.csv")
df_측정소_보간데이터2 = pd.read_csv("df_측정소_보간데이터2.csv")
df_측정소_보간데이터2['측정월'] = pd.to_datetime(pd.to_datetime(df_측정소_보간데이터2['날짜'])).dt.month
df_측정소_유동인구 = pd.read_csv("df_측정소_유동인구.csv")
df_측정소_유해시설 = pd.read_csv("df_측정소_유해시설.csv")
초미세먼지_train = pd.merge(df_측정소_보간데이터2, df_측정소_도로, on = '측정소', how = 'left').drop(['link_id', 'geo_cent_x','geo_cent_y'], axis = 1)
초미세먼지_train = pd.merge(초미세먼지_train, df_측정소_유동인구, on = ['측정소','측정월'], how = 'left')
초미세먼지_train = pd.merge(초미세먼지_train, df_측정소_유해시설, on = '측정소', how = 'left').drop('geometry_factory', axis = 1)

In [ ]:
print(초미세먼지_train.shape)
초미세먼지_train.head()

(2712, 19)


,날짜,측정소,target,보간데이터_4,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도,측정월,road_distance,교통량_12,교통량_17,lanes,TMST_10,TMST_18,factory_num,주차장여부
0,2020-01-01,고색동,22.600000,21.056205,0.2,2.4,90.0,-6.3,0.2,64.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
1,2020-01-02,고색동,49.615385,40.924386,3.2,1.4,92.0,-0.3,0.1,76.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
2,2020-01-03,고색동,51.050000,50.600915,5.2,3.0,99.0,-3.9,0.0,51.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
3,2020-01-04,고색동,43.375000,41.091517,7.4,3.4,100.0,-4.3,0.1,48.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1
4,2020-01-05,고색동,36.950000,36.200925,6.2,1.7,96.0,-5.5,0.3,43.0,1,0.000618,149.0,205.0,2,1330.0,1967.57,62,1


In [ ]:
초미세먼지_train.to_csv("초미세먼지_train.csv", index = False)

## b) 미세먼지 · 초미세먼지 예측데이터

### b-1) 미세먼지

In [ ]:
df_버스_도로 = pd.read_csv("df_버스_도로.csv")
df_버스_보간데이터 = pd.read_csv("df_버스_보간데이터.csv")
df_버스_보간데이터['월'] = pd.to_datetime(pd.to_datetime(df_버스_보간데이터['날짜'])).dt.month
df_버스_유동인구_시간 = pd.read_csv("df_버스_유동인구_시간.csv")
df_버스_유해시설 = pd.read_csv("df_버스_유해시설.csv")
미세먼지_test = pd.merge(df_버스_보간데이터, df_버스_도로[['정류소ID','12','17','road_distance','lanes']], on = '정류소ID', how = 'left')
미세먼지_test = pd.merge(미세먼지_test, df_버스_유동인구_시간, on = ['정류소ID','월'], how = 'left')
미세먼지_test = pd.merge(미세먼지_test, df_버스_유해시설, on = '정류소ID', how = 'left').drop('geometry_factory', axis = 1)
미세먼지_test.rename(columns = {'보간데이터':'보간데이터_4', '12':'교통량_12','17':'교통량_17','유동인구_10':'TMST_10','유동인구_18':'TMST_18'}, inplace = True) 

In [ ]:
print(미세먼지_test.shape)
미세먼지_test.head()

(243088, 18)


,날짜,정류소ID,target,보간데이터_4,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도,월,교통량_12,교통량_17,road_distance,lanes,TMST_10,TMST_18,factory_num
0,2020-01-01,201000055,0,37.005345,0.2,2.4,90.0,-6.3,0.2,64.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
1,2020-01-02,201000055,0,68.316946,3.2,1.4,92.0,-0.3,0.1,76.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
2,2020-01-03,201000055,0,80.674983,5.2,3.0,99.0,-3.9,0.0,51.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
3,2020-01-04,201000055,0,65.811928,7.4,3.4,100.0,-4.3,0.1,48.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
4,2020-01-05,201000055,0,57.449223,6.2,1.7,96.0,-5.5,0.3,43.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0


In [ ]:
미세먼지_test.to_csv("미세먼지_test.csv", index = False)

### b-2) 초미세먼지

In [ ]:
df_버스_도로=pd.read_csv("df_버스_도로.csv")
df_버스_보간데이터2=pd.read_csv("df_버스_보간데이터2.csv")
df_버스_보간데이터2['월']=pd.to_datetime(pd.to_datetime(df_버스_보간데이터2['날짜'])).dt.month
df_버스_유동인구_시간=pd.read_csv("df_버스_유동인구_시간.csv")
df_버스_유해시설=pd.read_csv("df_버스_유해시설.csv")
초미세먼지_test=pd.merge(df_버스_보간데이터2, df_버스_도로[['정류소ID','12','17','road_distance','lanes']], on='정류소ID', how='left')
초미세먼지_test=pd.merge(초미세먼지_test, df_버스_유동인구_시간, on=['정류소ID','월'], how='left')
초미세먼지_test=pd.merge(초미세먼지_test, df_버스_유해시설, on='정류소ID', how='left').drop('geometry_factory', axis=1)
초미세먼지_test.rename(columns={'보간데이터':'보간데이터_4', '12':'교통량_12','17':'교통량_17','유동인구_10':'TMST_10','유동인구_18':'TMST_18'}, inplace=True) # df_08과 동일하게

In [ ]:
print(초미세먼지_test.shape)
초미세먼지_test.head()

(243122, 18)


,날짜,정류소ID,target,보간데이터_4,최고기온,최고풍속,최고습도,최저기온,최저풍속,최저습도,월,교통량_12,교통량_17,road_distance,lanes,TMST_10,TMST_18,factory_num
0,2020-01-01,201000055,0,22.837865,0.2,2.4,90.0,-6.3,0.2,64.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
1,2020-01-02,201000055,0,43.541571,3.2,1.4,92.0,-0.3,0.1,76.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
2,2020-01-03,201000055,0,53.757842,5.2,3.0,99.0,-3.9,0.0,51.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
3,2020-01-04,201000055,0,44.082389,7.4,3.4,100.0,-4.3,0.1,48.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0
4,2020-01-05,201000055,0,37.471504,6.2,1.7,96.0,-5.5,0.3,43.0,1,477.0,607.5,0.000088,4,2692.44,3547.08,0


## c) 이용시간 학습데이터

In [ ]:
df_01 = pd.read_csv("input/1.수원시_버스정류장.csv")
df_02 = pd.read_csv("input/2.수원시_버스정류장별_승하차이력(1).csv")
df_03 = pd.read_csv("input/3.수원시_버스정류장별_승하차이력(2).csv")
df_04 = pd.read_csv("input/4.수원시_버스정류장별_승하차이력(3).csv")
df_05 = pd.read_csv("input/5.수원시_버스정류장별_승하차이력(4).csv")
df_06 = pd.read_csv("input/6.수원시_버스정류장별_승하차이력(5).csv")

df_버스_도로 = pd.read_csv("df_버스_도로.csv")
df_버스_유동인구_요일 = pd.read_csv("df_버스_유동인구_요일.csv")
df_버스_도로 = pd.read_csv("df_버스_도로.csv")
df_버스_간격 = pd.read_csv("df_버스_간격.csv")
df_버스_건물 = pd.read_csv("df_버스_건물.csv")
df_버스_노선개수 = pd.read_csv("df_버스_노선개수.csv")
df_버스_인구 = pd.read_csv('df_버스_인구.csv')
df_버스_유형 = pd.read_csv("df_버스_유형.csv")

이용시간_cluster = pd.merge(df_버스_도로[['정류소ID','혼잡시간강도','혼잡빈도강도','width']], df_버스_건물, on = '정류소ID', how = 'left')
이용시간_cluster = pd.merge(이용시간_cluster, df_버스_유형[['정류소ID','시외여부','마을여부','공항여부']], on = '정류소ID', how = 'left')
이용시간_cluster = pd.merge(이용시간_cluster, df_버스_인구, on = '정류소ID',how = 'left')
이용시간_cluster = pd.merge(이용시간_cluster, df_버스_노선개수, on = '정류소ID', how = 'left')

# df_33 = 버스 승하차 이력 전체 데이터 
df_33 = pd.concat([df_02, df_03, df_04, df_05, df_06], axis=0)
df_33 = df_33.drop_duplicates().reset_index()
df_33['date'] = df_33['일자'].apply(lambda x: pd.to_datetime(str(x)))
df_33['요일'] = df_33['date'].dt.weekday
df_33['주말여부'] = df_33['요일'].apply(lambda x: 0 if x<5 else 1)
df_33['월'] = df_33['date'].dt.month

df_33['노선번호'] = df_33['노선번호'].apply(lambda x: re.sub('_', '-', str(x)))
df_33.loc[(df_33['노선번호'] == '4000'),'업체명'] = '성우운수' # 4000번 버스의 업체명이 잘못 들어가있음
df_33['운행업체'] = df_33['업체명'].apply(lambda x: str(x).replace("(주)", ""))
df_33['운행업체'] = df_33['운행업체'].apply(lambda x: str(x).replace("버스라인", ""))
df_33['운행업체'] = df_33['운행업체'].replace('경진여객운수', '경진여객')

# 이용시간 column을 포함한 이용시간 예측모델의 최종 학습데이터
이용시간_train = pd.merge(df_33[['일자', '요일', '주말여부', '월', '노선번호', '정류소ID', '운행업체', '전체 승차 건수']], df_버스_간격[['노선번호', '운행업체', '최종배차', '주말여부']] ,on = ['노선번호', '운행업체', '주말여부'], how = 'right')
이용시간_train['이용시간'] = 이용시간_train['전체 승차 건수'] * 이용시간_train['최종배차']
이용시간_train = 이용시간_train.groupby(['일자','정류소ID','월','요일','주말여부']).sum().reset_index()
이용시간_train = pd.merge(이용시간_train, 이용시간_cluster, on = '정류소ID', how = 'inner')
이용시간_train = pd.merge(이용시간_train, df_버스_유동인구_요일, on = ['월','요일','정류소ID'], how = 'left')
이용시간_train = 이용시간_train.groupby(['정류소ID','월','요일']).mean().reset_index()
이용시간_train = 이용시간_train[이용시간_train['이용시간']!=0]

In [ ]:
print(이용시간_cluster.shape)
이용시간_cluster.head()

(698, 27)


,정류소ID,혼잡시간강도,혼잡빈도강도,width,학교_5분이내거리평균,집_5분이내거리평균,회사_5분이내거리평균,공장_5분이내거리평균,상업시설_5분이내거리평균,관공서_5분이내거리평균,...,지하철_5분이내개수,병원_5분이내거리평균,병원_5분이내개수,시외여부,마을여부,공항여부,인구수_유소년,인구수_생산가능,인구수_고령,노선개수
0,201000055,82.260,50.400,3,0.003338,0.003345,0.004040,0.0,0.00214,0.001598,...,0,0.004883,5,0,0,0,1457,12643,1711,6
1,201000268,48.760,30.810,5,0.002305,0.003015,0.002923,0.0,0.00000,0.002923,...,0,0.003269,1,0,0,0,1996,14255,1327,14
2,201000337,42.515,16.820,3,0.002560,0.003182,0.003980,0.0,0.00000,0.000000,...,0,0.000000,0,0,0,0,1491,6175,1046,8
3,202000089,42.075,17.305,4,0.003543,0.003973,0.003086,0.0,0.00000,0.004060,...,1,0.000000,0,0,1,0,1067,8335,1482,7
4,201000358,32.255,9.930,4,0.003874,0.002674,0.002534,0.0,0.00000,0.004786,...,0,0.004733,1,0,0,0,2230,10242,1394,14


In [ ]:
print(이용시간_train.shape)
이용시간_train.head()

(58292, 37)


,정류소ID,월,요일,일자,주말여부,전체 승차 건수,최종배차,이용시간,혼잡시간강도,혼잡빈도강도,...,시외여부,마을여부,공항여부,인구수_유소년,인구수_생산가능,인구수_고령,노선개수,lon,lat,유동인구
0,200000005,1,0,20200116.5,0.0,6.00,3.3,19.800,49.14,16.59,...,0.0,0.0,0.0,200.0,2567.0,755.0,1.0,127.031546,37.301462,43811.04
1,200000005,1,1,20200117.5,0.0,9.25,3.3,30.525,49.14,16.59,...,0.0,0.0,0.0,200.0,2567.0,755.0,1.0,127.031546,37.301462,55190.34
2,200000005,1,2,20200115.0,0.0,12.60,3.3,41.580,49.14,16.59,...,0.0,0.0,0.0,200.0,2567.0,755.0,1.0,127.031546,37.301462,19915.23
3,200000005,1,3,20200116.0,0.0,8.20,3.3,27.060,49.14,16.59,...,0.0,0.0,0.0,200.0,2567.0,755.0,1.0,127.031546,37.301462,56852.50
4,200000005,1,4,20200117.0,0.0,8.00,3.3,26.400,49.14,16.59,...,0.0,0.0,0.0,200.0,2567.0,755.0,1.0,127.031546,37.301462,41069.53


In [ ]:
이용시간_cluster.to_csv("이용시간_cluster.csv", index = False)
이용시간_train.to_csv("이용시간_train.csv", index = False)

## d) 이용시간 예측 데이터

In [ ]:
df_정류장정리 = pd.read_csv("df_버스_유동인구_요일.csv")
df_정류장별_유동인구 = df_정류장정리.sort_values(by=['정류소ID', '월', '요일']).reset_index(drop=True)

# 정규화 함수
def normalize(data, variable):
    return (data[variable] - np.min(data[variable])) / (np.max(data[variable])-np.min(data[variable]))

# 예측 데이터 형태 변환 ([과거 데이터 (lag개) | 현재 데이터])
def generate_timeseries(data, lag):
    X = []
    y = []
    for i in range(len(data)):
        x = data[i:(i + lag)]
        if (i + lag) < len(data):
            X.append(x)
            y.append(data[i + lag])
        else:
            break
    return np.array(X), np.array(y)

final_list = []
for bus in range(len(df_정류장별_유동인구.정류소ID.unique())):
    
    # 데이터셋 생성
    start_idx = bus * 84
    end_idx = (bus + 1) * 84
    data = df_정류장별_유동인구.iloc[start_idx:end_idx,:].reset_index(drop=True)
    data_max = np.max(data.유동인구); data_min = np.min(data.유동인구)
    bus_data = normalize(data, '유동인구'); lag = 5
    X, y = generate_timeseries(bus_data, lag)
    X = X.reshape(-1, lag, 1)
    y = y.reshape(-1, 1, 1)
    
    # Train, Test 분리
    X_train = X[:-1]; y_train = y[:-1]
    X_test = X[-1].reshape(-1,lag,1); y_test = y[-1].reshape(-1,1)
    
    # 모델 학습
    print('=============={}th bus-stop=============='.format(bus+1))
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(10))
    model.add(keras.layers.Dense(1))
    model.compile(loss=keras.losses.mean_squared_error, optimizer='rmsprop')
    model.fit(X_train, y_train, epochs = 5, batch_size = 2)
    
    # 모델 예측
    pred_month = 3
    pred_list = []
    final_pred_list = []
    for _ in range(7 * pred_month):
        pred = model.predict(X_test)
        final_pred = pred * (data_max - data_min) + data_min
        pred_list.append(pred)
        final_pred_list.append(final_pred)
        X_test = np.delete(np.concatenate([X_test, pred], axis=None), 0, 0).reshape(-1, lag, 1)
    final_pred_list = np.array(final_pred_list).flatten()
    final_list.append(final_pred_list)
    
df_버스_이용시간_요일_예측 = pd.DataFrame(columns = ['정류소ID', 'lon', 'lat', '월', '요일', '유동인구', '유동인구_예측'])

for bus in range(len(df_정류장별_유동인구.정류소ID.unique())):
    start_idx = bus * 84
    end_idx = bus * 84 + 21
    data = df_정류장별_유동인구.iloc[start_idx:end_idx,:].reset_index(drop=True)
    data = pd.concat([data, pd.DataFrame({'유동인구_예측' : final_list[0]})], axis = 1)
    df_버스_이용시간_요일_예측 = pd.concat([df_버스_이용시간_요일_예측, data], axis = 0)

df_버스_이용시간_요일_예측 = df_버스_이용시간_요일_예측.reset_index(drop=True)
df_버스_이용시간_요일_예측.to_csv("df_버스_이용시간_요일_예측.csv", index = False)

==============1th bus-stop==============
Epoch 1/5
39/39 [==============================] - 0s 8ms/step - loss: 0.1018
Epoch 2/5
39/39 [==============================] - 0s 7ms/step - loss: 0.0746
Epoch 3/5
39/39 [==============================] - 0s 5ms/step - loss: 0.0706
Epoch 4/5
39/39 [==============================] - 0s 7ms/step - loss: 0.0694
Epoch 5/5
39/39 [==============================] - 0s 6ms/step - loss: 0.0686
==============2th bus-stop==============
Epoch 1/5
39/39 [==============================] - 0s 10ms/step - loss: 0.0961
Epoch 2/5
39/39 [==============================] - 0s 7ms/step - loss: 0.0739
Epoch 3/5
39/39 [==============================] - 0s 6ms/step - loss: 0.0683
Epoch 4/5
39/39 [==============================] - 0s 6ms/step - loss: 0.0652
Epoch 5/5
39/39 [==============================] - 0s 7ms/step - loss: 0.0609
==============3th bus-stop==============
Epoch 1/5
39/39 [==============================] - 0s 10ms/step - loss: 0.1114
Epoch 2/5
39/39 [

In [ ]:
df_버스_이용시간_요일_예측 = pd.read_csv("df_버스_이용시간_요일_예측.csv")
df_버스_이용시간_요일_예측['주말여부'] = df_버스_이용시간_요일_예측['요일'].apply(lambda x : 0 if x < 5 else 1)
이용시간_cluster = pd.read_csv("이용시간_cluster.csv")
이용시간_test = pd.merge(df_버스_이용시간_요일_예측, 이용시간_cluster, on = '정류소ID',how = 'left')
이용시간_test.to_csv("이용시간_test.csv", index = False)